In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split

np.random.seed(123)

import acquire

# Goal: Find the main demographic driver for Telco churn.

<u> Demographics I'll be looking at:</u>

- Senior citizens
- Single customers
- Customers with Partners
- Customers with Dependants
- Customers with Partner & Dependants

<u> Subgroups to compare once main driver is found, if I want to get more specific: </u>
- Gender
- monthly charges
- tenure

### Notes: By looking at the above demographics I want to find:
1) Highest churned demographic group (of the above)
- Note: I will first need to look at the group as whole (all churned and which group had most churns)
- Note: Then I should consider churn rate based with each group (example: senior citizens that have and have not churned rate)

# Getting to know my data:

In [2]:
#I am using the excel version as it was easier to acquire from my computer
df= acquire.excel_file()
#looking at columns as there seems to be alot
df.columns

Index(['customer_id', 'gender', 'is_senior_citizen', 'partner', 'dependents',
       'phone_service', 'internet_service', 'contract_type', 'payment_type',
       'monthly_charges', 'total_charges', 'churn', 'tenure', 'is_auto',
       'is_female', 'has_churned', 'has_phone', 'has_internet',
       'has_phone_internet', 'partner_dependants', 'calculated_monthly_avg',
       'validate', 'phone_service_type', 'internet_service_type',
       'contract_type.1', 'rounded_tenure', 'service_type', 'tenure_bins'],
      dtype='object')

In [3]:
#checking out dytpes
df.dtypes

customer_id                object
gender                     object
is_senior_citizen         float64
partner                    object
dependents                 object
phone_service             float64
internet_service          float64
contract_type             float64
payment_type               object
monthly_charges           float64
total_charges             float64
churn                      object
tenure                    float64
is_auto                     int64
is_female                    bool
has_churned                  bool
has_phone                    bool
has_internet                 bool
has_phone_internet           bool
partner_dependants          int64
calculated_monthly_avg     object
validate                   object
phone_service_type         object
internet_service_type      object
contract_type.1            object
rounded_tenure              int64
service_type               object
tenure_bins                object
dtype: object

<b><u>Takeaways:</b></u>
- I can already see that later down the road, I am going to want to change certain columns from object to float or bool so that I can have numbers to work with in my models those A or B/Yes or No columns
- I know my target (y) will be churn...again, after exploring, I want to change that to a bool

In [8]:
#it looks like I will want to fillna the the total_charges. Because it is a total charge, I am going to use mean (or average) of the total charges to fill those in.
df.isnull().sum()

customer_id                0
gender                     0
is_senior_citizen          0
partner                    0
dependents                 0
phone_service              0
internet_service           0
contract_type              0
payment_type               0
monthly_charges            0
total_charges             11
churn                      0
tenure                     0
is_auto                    0
is_female                  0
has_churned                0
has_phone                  0
has_internet               0
has_phone_internet         0
partner_dependants         0
calculated_monthly_avg     0
validate                   0
phone_service_type         0
internet_service_type      0
contract_type.1            0
rounded_tenure             0
service_type               0
tenure_bins                0
dtype: int64

In [32]:
#let's fill in those NaNs
total_mean=df.total_charges.mean()
df['total_charges'].fillna(value=total_mean, inplace=True)

df.isnull().sum()

customer_id               0
gender                    0
is_senior_citizen         0
partner                   0
dependents                0
phone_service             0
internet_service          0
contract_type             0
payment_type              0
monthly_charges           0
total_charges             0
churn                     0
tenure                    0
is_auto                   0
is_female                 0
has_churned               0
has_phone                 0
has_internet              0
has_phone_internet        0
partner_dependants        0
calculated_monthly_avg    0
validate                  0
phone_service_type        0
internet_service_type     0
contract_type.1           0
rounded_tenure            0
service_type              0
tenure_bins               0
dtype: int64

In [14]:
#for senior citizen demographics: 
df.is_senior_citizen.value_counts()

#TAKE AWAYS: Senior citizens make up 19% of Telco customers

0.0    5907
1.0    1142
Name: is_senior_citizen, dtype: int64

In [6]:
#for clients with partner and/or dependants demographics: 
#note: 0= single |  3= has partner AND dependants | 2= has dependants  | 1= has partner
df.partner_dependants.value_counts()

#Takeaways: Singles represent 47% of Telco customers
          # Families represent 25% of Telco customers
          # Parteners represent 23% of Telco customers
          # Single parents represent 5% of Telco customers

0    3281
3    1754
1    1653
2     361
Name: partner_dependants, dtype: int64

In [7]:
#for gender demographics:
df.gender.value_counts()

#Takeaways: Males represent 50% of Telco customers
          # Females represent 50% of Telco customers

Male      3558
Female    3491
Name: gender, dtype: int64

## Next Steps:
- ✅ Creating range columns (for upcoming hypothesis testing) 
- Changing objects to floats for hypothesis testing
- Explore some charts of the data (histograms comparing demographics)

In [33]:
#creating the columns for range in the statistical table of df
##used this great function from Faith Kane
def col_range(df):
    stats_df = df.describe().T
    stats_df['range'] = stats_df['max'] - stats_df['min']
    return stats_df

In [39]:
col_range(df)

,count,mean,std,min,25%,50%,75%,max,range
is_senior_citizen,7049.0,0.162009,0.368485,0.00,0.000000,0.000000,0.000000,1.000000,1.000000
phone_service,7049.0,1.324585,0.642709,0.00,1.000000,1.000000,2.000000,2.000000,2.000000
internet_service,7049.0,1.222585,0.779068,0.00,1.000000,1.000000,2.000000,2.000000,2.000000
contract_type,7049.0,0.690878,0.833757,0.00,0.000000,0.000000,1.000000,2.000000,2.000000
monthly_charges,7049.0,64.747014,30.099460,18.25,35.450000,70.350000,89.850000,118.750000,100.500000
total_charges,7049.0,2283.043883,2264.752583,18.80,402.500000,1400.300000,3784.000000,8684.800000,8666.000000
tenure,7049.0,32.379866,24.595524,0.00,8.733456,28.683425,55.229399,79.341772,79.341772
is_auto,7049.0,0.435806,0.495897,0.00,0.000000,0.000000,1.000000,1.000000,1.000000
partner_dependants,7049.0,1.083416,1.226883,0.00,0.000000,1.000000,2.000000,3.000000,3.000000
rounded_tenure,7049.0,32.374805,24.596637,0.00,9.000000,29.000000,55.000000,79.000000,79.000000


In [96]:
# Preparing and Seperating the Data:
#- Separating the Splitting data to begin exploration and hypothesis testing
##note: this will be added to the prepare.py file
def telco_split(df):
    '''
    This function takes in Telco data from the acquire.py file,
    performs a split and stratifies on churn.
    Returns train, validate, and test dfs.
    '''
    train_validate, test = train_test_split(df, test_size=.2, 
                                        random_state=123, 
                                        stratify=df.churn)
    train, validate = train_test_split(train_validate, test_size=.3, 
                                   random_state=123, 
                                   stratify=train_validate.churn)
    return train, validate, test

In [97]:
def prep_telco(df):
    '''
    This function takes in the Telco df(via acquire.py).
    It drops the species_id column and renames species_name to species, 
    Performs a 3-way split stratified on churn, and
    Returns train, validate, and test dataframes.
    '''
    # drop and rename columns
    df = df.drop(columns='customer_id').rename(columns={'has_churned': 'Churned_Customers'})
    
    # split dataframe into train, validate, and test
    train, validate, test = telco_split(df)
    
    return train, validate, test

In [98]:
train, validate, test = prep_telco(df)

print(f'train -> {train.shape}')
print(f'validate -> {validate.shape}')
print(f'test -> {test.shape}')

train -> (3947, 27)
validate -> (1692, 27)
test -> (1410, 27)
